In [100]:
import numpy as np

In [161]:
# с семинара

num_companies = 4  # кол-во компаний
capital = 100  # в тыс. руб.
step = 10
profit_of_investment = [[0, 7, 13, 20, 33, 35],
                        [0, 8, 12, 21, 28, 40],
                        [0, 10, 11, 19, 30, 34],
                        [0, 6, 9, 24, 30, 36]]

In [102]:
# из моего варианта

num_companies = 4  # кол-во компаний
capital = 700  # в тыс. руб.
step = 100
profit_of_investment = [[0, 28, 45, 65, 78, 90, 102, 113],
                        [0, 25, 41, 55, 65, 75, 80, 85],
                        [0, 15, 25, 40, 50, 62, 73, 82],
                        [0, 20, 33, 42, 48, 53, 56, 58]]

In [110]:
U_matrix = [[u for u in range(0, min(capital + step, step * len(profit_of_investment[j])), step)] for j in
            range(0, len(profit_of_investment))]
U_matrix_indexes = [[U.index(u) for u in U] for U in U_matrix]
U_matrix_indexes

[[0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, 5]]

In [111]:
X_matrix = []
for row in range(0, len(profit_of_investment)):
    if row == len(profit_of_investment) - 1:
        max_capital = min(capital + step, step * len(profit_of_investment[len(profit_of_investment) - 1]))
    else:
        max_capital = capital + step
    X_row = [x for x in range(0, max_capital, step)]
    X_matrix.append(X_row)

X_matrix_indexes = [[X.index(x) for x in X] for X in X_matrix]
X_matrix_indexes

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [0, 1, 2, 3, 4, 5]]

In [153]:
# my test case

num_companies = 3  # кол-во компаний
capital = 50  # в тыс. руб.
step = 10
profit_of_investment = [[0, 1],
                        [0, 1, 2, 3],
                        [0, 1, 2]]

In [162]:
# считаем возможные управления (U_matrix)

U_matrix = []
for k in range(0, num_companies): # идем по предприятиям
    max_u_index = len(profit_of_investment[k])
    U = [i * step for i in range(max_u_index)]
    U_matrix.append(U)

U_matrix

[[0, 10, 20, 30, 40, 50],
 [0, 10, 20, 30, 40, 50],
 [0, 10, 20, 30, 40, 50],
 [0, 10, 20, 30, 40, 50]]

In [163]:
# считаем возможные состояния (X_matrix)

X_matrix = []
x_initial = capital
for k in range(0, num_companies): # идем по предприятиям
    X = []
    if k == 0:
        X = [x_initial - u for u in U_matrix[k]]
    else:
        # посчитаем все возможные состояния после того как подействуем управлением u(k) на x(k - 1)
        X = []
        for x in X_matrix[k - 1]:
            for u in U_matrix[k]:
                X.append(x - u)
        X = list(set(X)) # удалим дубликаты
        X = sorted(X) # отсортируем
        X = [x for x in X if x >= 0]# уберем отрицательные значения

        # min_u = min(U_matrix[k])
        # max_u = max(U_matrix[k])
        # min_prev_x = min(X_matrix[k - 1])
        # max_prev_x = max(X_matrix[k - 1])
        # X = [x for x in range (min_prev_x - max_u, max_prev_x)]
    X_matrix.append(X)

X_matrix

[[100, 90, 80, 70, 60, 50],
 [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]]

In [168]:
# итоговая функция

def foo(num_companies, capital, A: np.array):
    """
    :param num_companies: количество предприятий
    :param capital: капитал
    :param profit_of_investment: двумерный массив
    :return:
    """
    F = {}  # F[(k, i)] means (F_k(i), U_k)
    for k in range(num_companies - 1, -1, -1):  # идем от (num_companies - 1) до 0
        X = [i for i in X_matrix[k]]
        print("Расчет F(k = {0}) при X = {1}".format(k, X))
        for x in X:
            if k == num_companies - 1:
                if X.index(x) > len(A[k]) - 1:
                    w = 0
                else:
                    w = A[k][X.index(x)]
                u = x
                F[(k, x)] = [w, u]
            else:
                w_list = []
                for u in U_matrix[k]:
                    w_list.append(A[k][U.index(u)] + F[(k + 1, X.index(x) - U.index(u))][0])
                w = max(w_list)
                u = w_list.index(w)
                F[(k, x)] = [w, u]
            print("F({0}, {1}) = ".format(k, x), F[(k, x)])

    print("Answer = ", F[(0, capital / step)])

    for i in range(1):
        # print("Variant 1:")
        pass
        for j in range(1):
            # print("It should be spent (*amount1*) on company (*number1*)")
            # print("Total profit is (*target value*))")
            pass


foo(num_companies, capital, profit_of_investment)

Расчет F(k = 3) при X = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
F(3, 0) =  [0, 0]
F(3, 10) =  [6, 10]
F(3, 20) =  [9, 20]
F(3, 30) =  [24, 30]
F(3, 40) =  [30, 40]
F(3, 50) =  [36, 50]
F(3, 60) =  [0, 60]
F(3, 70) =  [0, 70]
F(3, 80) =  [0, 80]
F(3, 90) =  [0, 90]
F(3, 100) =  [0, 100]
Расчет F(k = 2) при X = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


KeyError: (3, -1)